# Verify GPU

In [1]:
!nvidia-smi

Sat Apr 24 18:22:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Weight and Bias (Assisting Metrics, Optional)

In [2]:
!pip install wandb
!wandb login
project_name = "CoLA with BERT"  # @param {type:"string"}
import os

os.environ["WANDB_PROJECT"] = project_name
import wandb

     |████████████████████████████████| 2.1MB 18.5MB/s 
     |████████████████████████████████| 133kB 60.6MB/s 
     |████████████████████████████████| 163kB 53.1MB/s 
     |████████████████████████████████| 102kB 14.3MB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=7bc528f9950f3b634b223b4a64022a13b85df1c246b140b0a4b6183eb7eee8ac
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=66adcb37901c9883a0ca6d62f9b60f611158288b35053bb378de1c4d44999742
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: 

# Installation

In [3]:
!pip install transformers==4.5.1 datasets==1.5.0 accelerate==0.2.1

     |████████████████████████████████| 2.1MB 17.6MB/s 
     |████████████████████████████████| 194kB 56.4MB/s 
     |████████████████████████████████| 51kB 8.9MB/s 
     |████████████████████████████████| 901kB 53.8MB/s 
     |████████████████████████████████| 3.3MB 54.7MB/s 
     |████████████████████████████████| 245kB 56.8MB/s 
     |████████████████████████████████| 112kB 59.6MB/s 


In [4]:
import os

import torch
from accelerate import Accelerator
from datasets import load_dataset, load_metric
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from transformers import (
    AdamW,
    BertConfig,
    BertForSequenceClassification,
    BertTokenizerFast,
    default_data_collator,
    get_linear_schedule_with_warmup,
)

In [5]:
per_device_train_batch_size = 32  # @param {type:"slider", min:1, max:64, step:1}
learning_rate = 2e-5  # @param {type:"slider", min:2e-7, max:2e-3, step:2e-7}
num_train_epochs = 3  # @param {type:"slider", min:1, max:10, step:1}
logging_steps = 10  # @param {type:"slider", min:10, max:100, step:5}

# Tokenizer

In [6]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")

# Dataset

In [7]:
datasets = load_dataset("glue", "cola")
label_list = datasets["train"].features["label"].names

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/cola/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4. Subsequent calls will reuse this data.


In [8]:
def preprocess_function(examples):
    # Tokenize the texts
    result = tokenizer(
        examples["sentence"],
        padding="max_length",
        max_length=128,
        truncation="longest_first",
    )
    result["label"] = examples["label"]
    return result

In [9]:
datasets = datasets.map(preprocess_function, batched=True)
datasets["test"] = datasets["test"].remove_columns("label")

In [10]:
for index in range(3):
    print(f"Sample {index} of the training set: {datasets['train'][index]}.")

Sample 0 of the training set: {'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'idx': 0, 'input_ids': [101, 3458, 2053, 1281, 112, 189, 4417, 1142, 3622, 117, 1519, 2041, 1103, 1397, 1141, 1195, 17794, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'label': 1, 'sentence': "Our friends won't buy this analysis, let alone the next one we propose.", 'toke

In [11]:
dataloaders = {
    "train": DataLoader(
        datasets["train"],
        collate_fn=default_data_collator,
        batch_size=per_device_train_batch_size,
        shuffle=True,
    ),
    "validation": DataLoader(
        datasets["validation"],
        collate_fn=default_data_collator,
        batch_size=per_device_train_batch_size,
        shuffle=True,
    ),
    "test": DataLoader(
        datasets["test"],
        collate_fn=default_data_collator,
        batch_size=per_device_train_batch_size,
        shuffle=True,
    ),
}

# Model

In [12]:
config = BertConfig.from_pretrained(
    "bert-base-cased", num_labels=len(label_list), finetuning_task="cola"
)
model = BertForSequenceClassification.from_pretrained("bert-base-cased", config=config)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

# Optimizer, scheduler, accelerator

In [13]:
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [
            p
            for n, p in model.named_parameters()
            if not any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.01,
    },
    {
        "params": [
            p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
num_warmup_steps = 0
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps, num_train_epochs * len(dataloaders["train"])
)

In [14]:
accelerator = Accelerator()
(
    model,
    optimizer,
    dataloaders["train"],
    dataloaders["validation"],
    dataloaders["test"],
) = accelerator.prepare(
    model,
    optimizer,
    dataloaders["train"],
    dataloaders["validation"],
    dataloaders["test"],
)

# Training

In [15]:
run_name = "Transformers Scratch"
if os.getenv("WANDB_PROJECT"):
    wandb.init(project=project_name, name=run_name)
    wandb.watch(model)
for epoch in range(num_train_epochs):
    model.train()
    for step, batch in enumerate(tqdm(dataloaders["train"])):
        batch.pop("idx")

        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)
        optimizer.step()
        scheduler.step()

        if step % logging_steps == 0:
            wandb.log(
                {
                    "train/loss": loss,
                    "train/epoch": epoch + step / len(dataloaders["train"]),
                    "train/learning_rate": scheduler.get_last_lr()[0],
                }
            )
    model.eval()
    metric = load_metric("glue", "cola")
    for batch in tqdm(dataloaders["validation"]):
        batch.pop("idx")

        outputs = model(**batch)
        outputs = accelerator.gather(outputs)
        predictions = outputs.logits.argmax(dim=-1)
        references = accelerator.gather(batch["labels"])

        metric.add_batch(
            predictions=predictions,
            references=references,
        )

    eval_metric = metric.compute()
    wandb.log({"eval/matthews_correlation": eval_metric["matthews_correlation"]})

os.mkdir(run_name)
accelerator.wait_for_everyone()
model = accelerator.unwrap_model(model)
accelerator.save(model.state_dict(), os.path.join(run_name, "trained_model.pt"))

wandb: Currently logged in as: tomy0000000 (use `wandb login --relogin` to force relogin)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:760: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


# Test

## On dataset

In [16]:
predictions = torch.empty(0, dtype=torch.int64, device=model.device)
for batch in tqdm(dataloaders["test"]):
    batch.pop("idx")
    outputs = model(**batch)
    predictions = torch.cat((predictions, outputs.logits.argmax(dim=-1)), 0)

In [17]:
for index, (sample, pred) in enumerate(zip(datasets["test"]["sentence"], predictions)):
    print(f"{index}\t{label_list[pred.item()]}\t{sample}")

0	acceptable	Bill whistled past the house.
1	unacceptable	The car honked its way down the road.
2	acceptable	Bill pushed Harry off the sofa.
3	acceptable	the kittens yawned awake and played.
4	acceptable	I demand that the more John eats, the more he pay.
5	acceptable	If John eats more, keep your mouth shut tighter, OK?
6	unacceptable	His expectations are always lower than mine are.
7	acceptable	The sooner you call, the more carefully I will word the letter.
8	acceptable	The more timid he feels, the more people he interviews without asking questions of.
9	acceptable	Once Janet left, Fred became a lot crazier.
10	acceptable	If you give every senator enough opportunity, he will succumb to corruption.
11	acceptable	The more time that any senator spends with lobbyists, the more likely he is to succumb to corruption.
12	unacceptable	Which problem does the sooner you solve, the more easily you'll satisfy the folks up at corporate headquarters?
13	unacceptable	Which problem do you think that t

## Manually

In [18]:
sentence = "The probable hostile German reaction is unfortunate."  # @param {type:"string"}
tokenized_input = tokenizer(sentence, return_tensors="pt").to(model.device)
outputs = model(**tokenized_input)
print(f"Prediction: {label_list[outputs.logits.argmax(dim=-1).item()]}")

Prediction: acceptable


# Inference

In [1]:
import torch
from transformers import BertConfig, BertForSequenceClassification, BertTokenizerFast

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
label_list = ["unacceptable", "acceptable"]
tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")
config = BertConfig.from_pretrained("bert-base-cased", finetuning_task="cola")
model = BertForSequenceClassification.from_pretrained(
    "Transformers Scratch/trained_model.pt", config=config
).to(device)

In [3]:
sentence = "The probable hostile German reaction is unfortunate."  # @param {type:"string"}
tokenized_input = tokenizer(sentence, return_tensors="pt").to(device)
outputs = model(**tokenized_input)
print(f"Prediction: {label_list[outputs.logits.argmax(dim=-1).item()]}")

Prediction: acceptable
